In [44]:
#This function finds out how long (in hours) a car is not moving within 10 kilometers
#at a specific point of time at 8:00 (base_time)
#This only works on grouped data farme in a for loop, not the lambda function
#This 

import pandas as pd
import datetime

data = pd.read_csv("")

#covert string to date time
times = pd.to_datetime(data['Breadcrumb Date UTC'], format= '%Y-%m-%d %H:%M:%S')
#append the times as utc_time column to the dataframe
data.loc[:, "utc_time"] = pd.Series(times , index = data.index)

df = data.loc[:,['AssetID','utc_time', 'City', 'Province','Country','Latitude','Longitude']]

#returning the row name, in this case indexes
def rowIndex(row):
    return row.name

row_index = df.apply(rowIndex, axis=1)

#adding indexes as keys to the data frame
df.loc[:,'row_index'] = row_index

base_time = datetime.datetime.strptime("2018-01-16 7:59", '%Y-%m-%d %H:%M')

import geopy
from geopy.distance import vincenty



def duration(base_index, data_frame):
    idle_dict = {}
    base_lat = data_frame.loc[base_index]['Latitude']
    base_lon = data_frame.loc[base_index]['Longitude']
    #return new data_frame for the rows where row_index > base_index (times are decending)
    #new_data_frame = data_frame.query('row_index > base_index')
    new_data_frame = data_frame[data_frame['row_index'] > base_index]
    lats = list(new_data_frame['Latitude'])
    lons = list(new_data_frame['Longitude'])
    coordinates = list(zip (lats, lons))
    coordinates_index = 0
    
    idle_dict[base_index] = data_frame.loc[base_index]['utc_time']
    
    for row in new_data_frame.itertuples(index = False):
        if(vincenty((base_lat, base_lon), coordinates[coordinates_index]).km < 10 ):
            idle_dict[row.row_index] = row.utc_time
            coordinates_index = coordinates_index + 1
    #idle_duraiton in seconds
    idle_duration = data_frame.loc[base_index]['utc_time'].timestamp()-idle_dict[max(idle_dict)].timestamp()
    # if duration is less than one hour
    if (idle_duration < 3600):
        return 0 
    else: #retun duration in seconds
        return (idle_duration/3600) 


#returns a tuple (row_index and the date of the nearest previous date compared to a specific point of time(baseDate))
def nearest_previous_index_and_date(baseDate, data_frame):
    dates = list(data_frame['utc_time'])
    nearness = {}
    for row in data_frame.itertuples(index = False):
        diff = baseDate.timestamp() - row.utc_time.timestamp()
        if(diff >= 0):
            nearness[diff] = (row.row_index, row.utc_time)
    return nearness[min(nearness.keys())] 


#return true if a car is idle at a specific point of time (base_time). 
#uses same City and 48 hours as idle criteria
def idle_car(base_date, data_frame):
    dates_ = data_frame['utc_time'].tolist()
    nearest_date = nearest_previous_date(base_date, dates_)
    nearest_48_h = nearest_previous_2_days(base_date, dates_)
    row_1 = data_frame[data_frame['utc_time'] == nearest_date]
    row_2 = data_frame[data_frame['utc_time'] == nearest_48_h]
    row_1.sort_index(axis=1).sort_index(inplace = True)
    row_2.sort_index(axis=1).sort_index(inplace = True)
    city_1 = row_1['City'].squeeze() #return tha scalar value of the dataframe
    city_2 = row_2['City'].squeeze()
    if (city_1 == city_2):
        return True
    else: 
        return False
    

#finding the idle status of each rail-car
results_2 = df.groupby('AssetID').apply(lambda x: idle_car(base_time, x)).reset_index(name="idle")
my_df = df[df['AssetID'] == 'CGAX9440']
#[] access works for one single group, not inside the lambda function
#print(nearest_previous_index_and_date(base_time, my_df)[0])
#print(nearest_previous_index_and_date(base_time, my_df)[1])

grouped_df = df.groupby('AssetID')
#results_3 = grouped_df.apply( lambda x: duration( nearest_previous_date_index(base_time, x)[0], x)).reset_index(name="idle-duration-hours")
#results_3

id_idle_duration = {}
for name, group in grouped_df:
    group_index_date_base = nearest_previous_index_and_date(base_time, grouped_df.get_group(name))
    id_idle_duration[name] = duration(group_index_date_base[0], grouped_df.get_group(name))
    

id_idle_duration

#returns the dictionary of the cars that are idle, idle duration is more than 48 hours
def is_idle(cars_dict):
    idle_cars = {}
    for key, val in cars_dict.items():
        if (val > 48):
            idle_cars[key] = val
    return idle_cars

final_res = is_idle(id_idle_duration)
final_res
len(grouped_df)
#index = 'idle_hours'
mm = pd.DataFrame.from_dict(final_res,orient= 'index' )#, index = index_)#dtype=None)
mm

,0
CGAX9054,218.933889
CGEX1480,61.233056
CGEX1584,61.022222
CGEX1787,60.810278
